In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [6]:
np.ceil(38*np.cumsum(np.array([0.7,0.2,0.1])))

array([27., 35., 38.])

In [ ]:
data = pd.read_csv('data/data.csv', parse_dates=['kickoff_time'])

In [ ]:
cutoff = 35

data_mid = data[(data['position'] == 'FWD')].reset_index(drop=True)

train_data = data_mid[data_mid['round'] < cutoff].drop('round', axis=1)
test_data = data_mid[data_mid['round'] >= cutoff].drop('round', axis=1)



wrangling_columns = ['element']

no_scaling = [
    'was_home', 'clean_sheets', 'yellow_cards', 'red_cards',
    'starts']

bounded_vars = ['minutes', 'bonus']  # minmax scaling

out_metrics = [
    'penalties_missed', 'goals_scored', 'assists', 'expected_goals',
    'expected_assists', 'expected_goal_involvements']

gkp_metrics = ['penalties_saved', 'saves']

count_vars = ['goals_conceded', 'own_goals']

fantasy_metrics = [
    'bps', 'influence', 'creativity', 'threat', 'ict_index', 'value',
    'transfers_balance', 'selected', 'transfers_in', 'transfers_out', 'total_points']

x_metrics = ['expected_goals_conceded']

time_metrics = ['time_diffs']

In [ ]:
scaler = MinMaxScaler().set_output(transform='pandas')

scaler_y =  MinMaxScaler()
scaler_y.fit(train_data[['total_points']])

min_max_vars = bounded_vars + out_metrics + count_vars + fantasy_metrics + x_metrics + time_metrics

scaler.fit(train_data[min_max_vars])
train_data = train_data[no_scaling + wrangling_columns].join(scaler.transform(train_data[min_max_vars]))
test_data = test_data[no_scaling + wrangling_columns].join(scaler.transform(test_data[min_max_vars]))

In [ ]:
window_size = 3

def get_window_arrays(data):

    X, y = [], []
    for player in tqdm(data['element'].unique(), desc='creating frames'):

        player_data = data[
            data['element'] == player].drop('element', axis=1).reset_index(
            drop=True).copy()

        frame_generator = player_data.drop(
            player_data.tail(1).index).rolling(window=window_size)

        for window in frame_generator:

            if len(window) >= window_size:
                X.append(np.array(window, float))

                points_index = window.index[-1]

                y.append(player_data['total_points'].loc[points_index + 1])

    return np.array(X), np.array(y, float)

X_train, y_train = get_window_arrays(train_data)
X_test, y_test = get_window_arrays(test_data)

np.save('data/X_train', X_train)
np.save('data/y_train', y_train)
np.save('data/X_test', X_test)
np.save('data/y_test', y_test)

X_test.shape


In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [ ]:



model = models.Sequential()
model.add(layers.Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(window_size, 28)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))



model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0)
preds = np.transpose(model.predict(X_test))[0]


In [ ]:

plt.figure(figsize=(30,5))

y_test_c = scaler_y.inverse_transform(pd.DataFrame(y_test)).transpose()[0]

preds_c = scaler_y.inverse_transform(pd.DataFrame(preds)).transpose()[0]

# plt.plot(np.roll(y_test, 1), 'b', marker='.')
plt.scatter(np.arange(len(preds_c)), preds_c, marker='x', color='r')
plt.scatter(np.arange(len(preds_c)), np.roll(y_test_c, 1),)
plt.plot( y_test_c, 'orange', marker='.')
# plt.scatter(y_test, 'g', alpha=0.5)

plt.show()

In [ ]:
np.ones(len(preds)) + 1

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test_c, preds_c, squared=False))

mean_squared_error(y_test_c,np.roll(y_test_c, 1), squared=False)

In [ ]:
def moving_average(a, n=10) :
    ret = np.cumsum(np.abs(a), dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


plt.figure(figsize=(20,10))

plt.plot(moving_average(np.roll(y_test, 1)-y_test), 'b', marker='.')
plt.plot(moving_average(preds-y_test), 'r', alpha=0.5)
plt.plot(moving_average(y_test-y_test), 'r', alpha=0.5)

plt.show()

In [ ]:



model = models.Sequential()
model.add(layers.Conv1D(filters=28, kernel_size=2, activation='relu', input_shape=(window_size, 28)))
model.add(layers.Conv1D(filters=28, kernel_size=2, activation='relu'))

model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0)
preds = np.transpose(model.predict(X_test))[0]

In [ ]:
model.summary()

In [ ]:
model.summary()

In [ ]:
data[(data['position'] == 'GKP') & (data['minutes'] >= 0)].plot.scatter(x='creativity', y='total_points')

In [ ]:
total_points = data['transfers_balance']

total_points.plot.hist(bins=1000)